In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls

gdrive	sample_data


In [2]:
import os
os.chdir("/content/gdrive/MyDrive/coggle_learn/learn")

In [ ]:
!mkdir model

In [3]:
import scipy.io as sio
import pandas as pd
import numpy as np
#from gensim.models import word2vec
from sklearn import preprocessing
from sklearn import metrics
import os
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.autograd import Variable
import time
import warnings
warnings.filterwarnings("ignore")
import random
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(2021)

In [4]:
path = './'
df = pd.read_csv(path+'trainreference.csv')

In [5]:
from torch.utils.data import Dataset, DataLoader
setup_seed(2021)
class myDataset(Dataset):
    def __init__(self, df, idx=None, if_train=True):
        self.if_train = if_train
        if self.if_train:
            self.paths = df.loc[idx, 'name'].reset_index(drop=True)
            self.labels = df.loc[idx, 'tag'].reset_index(drop=True)
        else:
            self.paths = df['name'].reset_index(drop=True)
            self.labels = df['tag'].reset_index(drop=True)

    def __getitem__(self, index):
        if self.if_train:
            sample = sio.loadmat(path+'train/'+self.paths[index])['ecgdata']
        else:
            sample = sio.loadmat(path+'val/'+self.paths[index])['ecgdata']
        return sample, self.labels[index]

    def __len__(self):
        return len(self.paths)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SeqNet(nn.Module):
    def __init__(self):
        super(SeqNet, self).__init__()
        # input 
        self.conv1 = nn.Conv1d(12, 10, 50)
        self.conv2 = nn.Conv1d(12, 10, 200)
        self.conv3 = nn.Conv1d(12, 10, 500)
        self.conv4 = nn.Conv1d(12, 10, 1000)
        self.pooling = nn.MaxPool2d((1, 200))
        self.fc1 = nn.Linear(900, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        batch_size = x.size(0)
        
        out1 = self.pooling(F.relu(self.conv1(x)))
        out2 = self.pooling(F.relu(self.conv2(x)))
        out3 = self.pooling(F.relu(self.conv3(x)))
        out4 = self.pooling(F.relu(self.conv4(x)))

        out = torch.cat([out1, out2, out3, out4], 2)
        out = out.view(batch_size, -1)
        out = self.fc1(out)
        out = F.relu(out)
        # out = F.dropout(out, p=0.2)
        out = self.fc2(out)
        return out

In [7]:
trainloader = torch.utils.data.DataLoader(
        myDataset(df, [i for i in range(0, 100)]), 
        batch_size=30, shuffle=True, pin_memory=True, num_workers=1)

In [8]:
for i in trainloader:
    x = i[0]
    y = i[1]
    break
type(x)

torch.Tensor

In [13]:
x.shape

torch.Size([30, 12, 5000])

In [15]:
conv1 = nn.Conv1d(12, 10, 50)
conv2 = nn.Conv1d(12, 10, 200)
conv3 = nn.Conv1d(12, 10, 500)
conv4 = nn.Conv1d(12, 10, 1000)
pooling = nn.MaxPool2d((1, 200))
fc1 = nn.Linear(900, 64)
fc2 = nn.Linear(64, 1)

In [23]:
print('数据shape:', x.shape)
conv1 = nn.Conv1d(12, 10, 50)
pooling = nn.MaxPool2d((1,200))
print(pooling)
m = F.relu(conv1(x))
print(m.shape)
pool_out = pooling(m)
pool_out.shape

数据shape: torch.Size([30, 12, 5000])
MaxPool2d(kernel_size=(1, 200), stride=(1, 200), padding=0, dilation=1, ceil_mode=False)
torch.Size([30, 10, 4951])


torch.Size([30, 10, 24])

In [25]:
 m = nn.MaxPool2d(3, stride=2)
# pool of non-square window
m = nn.MaxPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m(input)
output.shape

torch.Size([20, 16, 24, 31])

In [9]:
model = SeqNet()

print(model(x))

tensor([[ 0.0957],
        [ 0.0948],
        [ 0.0925],
        [ 0.1222],
        [ 0.0994],
        [ 0.0951],
        [ 0.0951],
        [ 0.1369],
        [ 0.0655],
        [ 0.1299],
        [ 0.1154],
        [ 0.1094],
        [ 0.1008],
        [ 0.1027],
        [ 0.0954],
        [ 0.0892],
        [ 0.1026],
        [ 0.1097],
        [-0.0969],
        [ 0.1188],
        [ 0.0831],
        [ 0.1217],
        [ 0.0923],
        [ 0.1131],
        [ 0.1148],
        [ 0.1390],
        [ 0.0688],
        [ 0.1194],
        [ 0.1042],
        [ 0.0961]], grad_fn=<AddmmBackward0>)


In [ ]:
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
max_epoch = 20
model_save_dir = path+'model/'
def train_model(model, criterion, optimizer, lr_scheduler=None):
    total_iters=len(trainloader)
    print('--------------total_iters:{}'.format(total_iters))
    since = time.time()
    best_loss = 1e7
    best_epoch = 0
    best_f1 = 0
    #
    iters = len(trainloader)
    for epoch in range(1,max_epoch+1):
        model.train(True)
        begin_time=time.time()
        # print('learning rate:{}'.format(optimizer.param_groups[-1]['lr']))
        print('Fold{} Epoch {}/{}'.format(fold+1,epoch, max_epoch))
        running_corrects_linear = 0
        count=0
        train_loss = []
        for i, (inputs, labels) in (enumerate(trainloader)):
            # print(inputs)
            count+=1
            inputs = inputs.to(device)
            labels = labels.float().to(device)

            out_linear = model(inputs).to(device)
            loss = criterion(out_linear, labels.unsqueeze(1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # 更新cosine学习率
            if lr_scheduler!=None:
                lr_scheduler.step(epoch + count / iters)
            if print_interval>0 and (i % print_interval == 0 or out_linear.size()[0] < train_batch_size):
                spend_time = time.time() - begin_time
                print(
                    ' Fold:{} Epoch:{}({}/{}) loss:{:.3f} lr:{:.7f} epoch_Time:{}min:'.format(
                        fold+1,epoch, count, total_iters,
                        loss.item(), optimizer.param_groups[-1]['lr'],
                        spend_time / count * total_iters // 60 - spend_time // 60))
            #
            train_loss.append(loss.item())
        #lr_scheduler.step()
        val_f1, val_loss= val_model(model, criterion)
        print('valf1: {:.4f}  valLogLoss: {:.4f}'.format(val_f1, val_loss))
        model_out_path = model_save_dir+"/"+'fold_'+str(fold+1)+'_'+str(epoch) + '.pth'
        best_model_out_path = model_save_dir+"/"+'fold_'+str(fold+1)+'_best'+'.pth'
        #save the best model
        if val_f1 >= best_f1:
            best_loss = val_loss
            best_f1 = val_f1
            best_epoch=epoch
            torch.save(model.state_dict(), best_model_out_path)
            print("save best epoch: {} best f1: {:.5f} best logloss: {:.5f}".format(best_epoch,val_f1,val_loss))
  
    print('Fold{} Best f1: {:.3f} Best logloss: {:.3f} Best epoch:{}'.format(fold+1,best_f1, best_loss,best_epoch))
    time_elapsed = time.time() - since
    return best_loss, best_f1

@torch.no_grad()
def val_model(model, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    cont = 0
    outPre = []
    outLabel = []
    pres_list=[]
    labels_list=[]
    for data in val_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        pres_list+=outputs.sigmoid().detach().cpu().numpy().tolist()
        labels_list+=labels.detach().cpu().numpy().tolist()

    preds = np.array(pres_list)
    labels = np.array(labels_list)
    val_f1 = metrics.f1_score(labels, list(map(lambda x: 1 if x > 0.5 else 0, preds)))
    log_loss = metrics.log_loss(labels, preds)#
    return val_f1, log_loss

In [ ]:
setup_seed(2021)
skf = StratifiedKFold(n_splits=10)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
criterion = nn.BCEWithLogitsLoss()
print_interval=-1
kfold_best_loss = []
kfold_best_f1 = []
# print(len(df))
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['tag'].values)):
    trainloader = torch.utils.data.DataLoader(
        myDataset(df, train_idx), 
        batch_size=32, shuffle=True, pin_memory=True, num_workers=1)
    val_loader = torch.utils.data.DataLoader(
        myDataset(df, val_idx), 
        batch_size=128, shuffle=False, pin_memory=True, num_workers=1)
    model = SeqNet()
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4 ,weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epoch)

    best_loss, best_acc = train_model(model, criterion, optimizer, lr_scheduler=scheduler)
    kfold_best_loss.append(best_loss)
    kfold_best_f1.append(best_acc)

print(kfold_best_f1)                  
print('loss...', np.mean(kfold_best_loss), 'f1...', np.mean(kfold_best_f1))

cuda
--------------total_iters:45
Fold1 Epoch 1/20
valf1: 0.6579  valLogLoss: 0.6611
save best epoch: 1 best f1: 0.65789 best logloss: 0.66112
Fold1 Epoch 2/20
valf1: 0.7361  valLogLoss: 0.5947
save best epoch: 2 best f1: 0.73611 best logloss: 0.59467
Fold1 Epoch 3/20
valf1: 0.7083  valLogLoss: 0.4831
Fold1 Epoch 4/20
valf1: 0.7639  valLogLoss: 0.4511
save best epoch: 4 best f1: 0.76389 best logloss: 0.45106
Fold1 Epoch 5/20
valf1: 0.8054  valLogLoss: 0.4288
save best epoch: 5 best f1: 0.80537 best logloss: 0.42876
Fold1 Epoch 6/20
valf1: 0.8075  valLogLoss: 0.4211
save best epoch: 6 best f1: 0.80745 best logloss: 0.42114
Fold1 Epoch 7/20
valf1: 0.7867  valLogLoss: 0.4153
Fold1 Epoch 8/20
valf1: 0.8000  valLogLoss: 0.4114
Fold1 Epoch 9/20
valf1: 0.8121  valLogLoss: 0.4157
save best epoch: 9 best f1: 0.81212 best logloss: 0.41567
Fold1 Epoch 10/20
valf1: 0.8054  valLogLoss: 0.3863
Fold1 Epoch 11/20
valf1: 0.8105  valLogLoss: 0.3839
Fold1 Epoch 12/20
valf1: 0.7838  valLogLoss: 0.3921
Fol

In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
def load_model(weight_path):
    print(weight_path)
    model = SeqNet()
    model.load_state_dict(torch.load(weight_path))
    model.to(device)
    model.eval()
    return model

@torch.no_grad()
def predict(test_loader):
    ret = 0
    for i, model in enumerate(model_list):
        print('----model ', i)
        pres_list = []
        for data in tqdm(test_loader):
            inputs, _a = data
            inputs = inputs.cuda()
            outputs = model(inputs)
            pres_list+=outputs.sigmoid().detach().cpu().numpy().tolist()
        ret += np.array(pres_list) / len(model_list)
    return list(map(lambda x: 1 if x > 0.5 else 0, ret))

In [ ]:
device=torch.device('cuda')
model_list=[]
for i in range(10):
    model_list.append(load_model(path+'model/fold_'+str(i+1)+'_best.pth'))
import os

sub = pd.read_csv(path+'answer.csv')
test_loader = torch.utils.data.DataLoader(
        myDataset(sub, if_train=False), 
        batch_size=64, shuffle=False, num_workers=16, pin_memory=True)
sub['tag'] = predict(test_loader)


./model/fold_1_best.pth
./model/fold_2_best.pth
./model/fold_3_best.pth
./model/fold_4_best.pth
./model/fold_5_best.pth
./model/fold_6_best.pth
./model/fold_7_best.pth
./model/fold_8_best.pth
./model/fold_9_best.pth
./model/fold_10_best.pth
----model  0


100%|██████████| 7/7 [00:18<00:00,  2.67s/it]


----model  1


100%|██████████| 7/7 [00:03<00:00,  2.33it/s]


----model  2


...

100%|██████████| 7/7 [00:02<00:00,  2.38it/s]


----model  8


100%|██████████| 7/7 [00:02<00:00,  2.37it/s]


----model  9


100%|██████████| 7/7 [00:02<00:00,  2.43it/s]


In [ ]:
# sub.head()
!ls answer

sub_20211118_0.83244.csv


In [ ]:
# !mkdir answer
sub.to_csv(path+'answer/answer_1124.csv', index=False)

In [ ]:
!zip model.zip model/*

  adding: model/fold_10_best.pth (deflated 8%)
  adding: model/fold_1_best.pth (deflated 8%)
  adding: model/fold_2_best.pth (deflated 8%)
  adding: model/fold_3_best.pth (deflated 8%)
  adding: model/fold_4_best.pth (deflated 8%)
  adding: model/fold_5_best.pth (deflated 8%)
  adding: model/fold_6_best.pth (deflated 8%)
  adding: model/fold_7_best.pth (deflated 8%)
  adding: model/fold_8_best.pth (deflated 8%)
  adding: model/fold_9_best.pth (deflated 8%)


In [ ]:
!ls

answer	    model      train	 trainreference.csv  Untitled0.ipynb  val.7z
answer.csv  model.zip  train.7z  train.zip	     val
